In [156]:
import numpy as np
import random
import math
import operator
import matplotlib.pyplot as plt
from pyts.datasets import fetch_ucr_dataset
from pyts.metrics import dtw
from saxpy.znorm import znorm
from saxpy.paa import paa
from saxpy.sax import ts_to_string
from saxpy.alphabet import cuts_for_asize


def SAX(result,i):
    dat = result[i]
    st = ts_to_string(dat,cuts_for_asize(5))
    return st

def pyo(p,q,r):
    list = []
    Gaus_dist_tup = {}
    Gaus_dist_tup[3] = [-0.43, 0.43]
    Gaus_dist_tup[4] = [-0.67, 0, 0.67]
    Gaus_dist_tup[5] = [-0.84, -0.25, 0.25, 0.84]
    Gaus_dist_tup[6] = [-0.97, -0.43, 0, 0.43, 0.97]
    Gaus_dist_tup[7] = [-1.07, -0.57, -0.18, 0.18, 0.57, 1.07]
    Gaus_dist_tup[8] = [-1.15, -0.67, -0.32, 0, 0.32, 0.67, 1.15]
    Gaus_dist_tup[9] = [-1.22, -0.76, -0.43, -0.14, 0.14, 0.43, 0.76, 1.22]
    Gaus_dist_tup[10] = [-1.28, -0.84, -0.52, -0.25, 0, 0.25, 0.52, 0.84, 1.28]
    for i in range(p):
        list.append([])
        for j in range(p):
            if i-j == 0 or i-j == 1 or i-j == -1:
                list[i].append(0)
            elif i > j:
                list[i].append(abs(Gaus_dist_tup[p][i-1]-Gaus_dist_tup[p][j]))
            elif i < j:
                list[i].append(abs(Gaus_dist_tup[p][j-1]-Gaus_dist_tup[p][i]))
    return list[q][r]
def PAA(X_train,n):
    x = 0
    result = []
    for i in range(len(X_train)):
        if i % n == 0 and i != 0:
           result.append(x/n)
           x = 0
        x = x + X_train[i]
    return result

def Distance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        a = pyo(10,instance1[x],instance2[x])
        distance += a
    return distance

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    traine = []
    teste = []

    #for i in range(len(trainingSet)):
        
    length = len(testInstance) - 1
    for x in range(len(trainingSet)):
        dist = Distance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors


def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]


def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct / float(len(testSet))) * 100.0


def main():
    X_train, X_test, y_train, y_test = fetch_ucr_dataset('FaceFour', return_X_y=True)
    XTrain = [0]*len(X_train)
    XTest = [0]*len(X_test)
    a = [0]*len(X_train)
    b = [0]*len(X_test)
    c = []
    d = []
    
    for i in range(len(X_train)):
        XTrain[i] = PAA(X_train[i],5)
        a[i] = SAX(XTrain,i)
    for i in range(len(X_test)):
        XTest[i] = PAA(X_test[i], 5)
        b[i] = SAX(XTest,i)
    for p in range(len(a)):
        c.append([])
        for q in range(len(a[p])):
            c[p].append(ord(a[p][q]) - ord('a'))
    for p in range(len(b)):
        d.append([])
        for q in range(len(b[p])):
            d[p].append(ord(b[p][q]) - ord('a'))
    trainingSet = np.c_[c, y_train]
    testSet = np.c_[d, y_test]
    predictions = []
    k = 1
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')


main()

> predicted=3, actual=3
> predicted=1, actual=1
> predicted=4, actual=2
> predicted=4, actual=4
> predicted=1, actual=1
> predicted=1, actual=4
> predicted=3, actual=3
> predicted=3, actual=3
> predicted=2, actual=2
> predicted=3, actual=3
> predicted=1, actual=1
> predicted=3, actual=3
> predicted=2, actual=2
> predicted=2, actual=2
> predicted=2, actual=3
> predicted=4, actual=2
> predicted=4, actual=2
> predicted=1, actual=3
> predicted=2, actual=2
> predicted=2, actual=4
> predicted=1, actual=1
> predicted=2, actual=2
> predicted=2, actual=2
> predicted=1, actual=1
> predicted=2, actual=2
> predicted=4, actual=4
> predicted=3, actual=3
> predicted=2, actual=2
> predicted=3, actual=3
> predicted=4, actual=4
> predicted=2, actual=3
> predicted=2, actual=2
> predicted=3, actual=3
> predicted=4, actual=4
> predicted=2, actual=3
> predicted=4, actual=4
> predicted=1, actual=1
> predicted=3, actual=3
> predicted=2, actual=4
> predicted=3, actual=3
> predicted=2, actual=2
> predicted=3, a